In [4]:
data = {"value1":'Test Title',
       "value2": "https://cdn.britannica.com/15/201015-050-E24FA163/Field-hockey-match-Poland-Spain-2017.jpg"}

In [5]:
requests.post("https://maker.ifttt.com/trigger/hockey/with/key/-qGdCTJMyPbzFAFLIZp-x", data=data)

<Response [200]>

In [1]:
import json
import requests
import plotly.graph_objects as go
from datetime import datetime
from requests_hawk import HawkAuth
import urllib.parse

In [121]:

def ordinal(n):
    if 11 <= n <= 13:
        return '{}th'.format(n)
    if n%10 == 1:
        return '{}st'.format(n)
    elif n%10 == 2:
        return '{}nd'.format(n)
    elif n%10 == 3:
        return '{}rd'.format(n)
    else:
        return '{}th'.format(n)
    
def my_format(ds):
    dt = datetime.strptime(ds, '%Y-%m-%d')
    return '{} {}'.format(dt.strftime('%A, %B'), ordinal(dt.day))

def get_color(colorscale_name, loc):
    from _plotly_utils.basevalidators import ColorscaleValidator
    # first parameter: Name of the property being validated
    # second parameter: a string, doesn't really matter in our use case
    cv = ColorscaleValidator("colorscale", "")
    # colorscale will be a list of lists: [[loc1, "rgb1"], [loc2, "rgb2"], ...] 
    colorscale = cv.validate_coerce(colorscale_name)
    
    if hasattr(loc, "__iter__"):
        return [get_continuous_color(colorscale, x) for x in loc]
    return get_continuous_color(colorscale, loc)
        

# Identical to Adam's answer
import plotly.colors
from PIL import ImageColor

def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
    
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    hex_to_rgb = lambda c: "rgb" + str(ImageColor.getcolor(c, "RGB"))

    if intermed <= 0 or len(colorscale) == 1:
        c = colorscale[0][1]
        return c if c[0] != "#" else hex_to_rgb(c)
    if intermed >= 1:
        c = colorscale[-1][1]
        return c if c[0] != "#" else hex_to_rgb(c)

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    if (low_color[0] == "#") or (high_color[0] == "#"):
        # some color scale names (such as cividis) returns:
        # [[loc1, "hex1"], [loc2, "hex2"], ...]
        low_color = hex_to_rgb(low_color)
        high_color = hex_to_rgb(high_color)

    return plotly.colors.find_intermediate_color(
        lowcolor=low_color,
        highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb",
    )

headers = {
    'authority': 'www.stubhub.ca',
    'pragma': 'no-cache',
    'cache-control': 'no-cache',
    'sec-ch-ua': '^\\^Google',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '^\\^Windows^\\^',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'accept': 'application/json',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'referer': 'https://www.google.com/',
    'accept-language': 'fr-FR,fr;q=0.9',
}

response = requests.get('https://www.stubhub.ca/fr/', headers=headers)


key = response.headers['Set-Cookie'].split("SH_BAU=")[1].split(';')[0]
ids =json.loads(urllib.parse.unquote(key))
hawk_auth = HawkAuth(id=ids['id'], key=ids['key'],always_hash_content=False)

In [122]:
startDate = "2021-10-13"
endDate = "2021-12-31"

url = "https://www.stubhub.ca/bfx/api/search/catalog/events/v3?shstore=8&status=active%20%7Ccontingent&spellCheck=true&boostByCategory=true&lang=true&includeNonEventEntities=true&edgeControlEnabled=true&fieldList=id%2CticketInfo%2Cdistance%2Cname%2CeventDateLocal%2Ccategories%2Cgroupings%2Cperformers%2Cvenue%2CeventInfoUrl%2CdisplayAttributes%2Cdescription%2CimageUrl%2CcreatedDate%2CperformersCollection%2CeventType%2CurgencyMessage%2CmetaInfo%2Cstatus&sourceId=0%20%7C1%20%7C4001%20%7C5001%20%7C29001&parking=false&point=45.50884%2C-73.58781&radius=50&excludeFromRadius=false&geoExpansion=false&start=0&rows=200&dateLocal=&sort=eventDateLocal%20asc&reRankBy=relevance&improvedRelevancy=true&eventType=Main%7CFestival%7CSeason%7CTailgate%7CHospitality&performerRole=homeTeam&performerId=7554"
r = requests.get(url, auth=hawk_auth, headers=headers)
j = json.loads(r.text)['events']
eventIDstubhub = {}
for e in j :
    eventIDstubhub[e["eventDateLocal"][:10]] = e["id"]


url = "https://statsapi.web.nhl.com/api/v1/schedule?site=fr_nhlCA&startDate="+startDate+"&endDate="+endDate+"&teamId=8&expand=schedule.teams,schedule.venue,schedule.metadata,schedule.ticket,schedule.broadcasts.all"
t = requests.get(url)
r = json.loads(t.text)


for d in r['dates']:
    if(d['games'][0]['teams']['home']['team']['name'] == 'Canadiens de Montréal'):
        tlink = d['games'][0]['tickets'][0]["ticketLink"]
        gameID = tlink.split("?")[0].split('/')[-1]
        urlPrice = "https://offeradapter.ticketmaster.com/api/ismds/event/"+gameID+"/facets?show=totalpricerange+places&by=offers&oq=not(locked)&q=available&apikey=b462oi7fic6pehcdkzony5bxhe&apisecret=pquzpfrfz7zd2ylvtz3w5dtyse&resaleChannelId=internal.ecommerce.consumer.desktop.web.browser.ticketmaster.ca"
        urlPlace = "https://mapsapi.tmol.io/maps/geometry/3/event/"+gameID+"/placeDetailNoKeys?systemId=HOST&useHostGrids=true&app=CCP&sectionLevel=true"
        prices = []
        r2 = json.loads(requests.get(urlPrice).text)['facets']
        rplace = json.loads(requests.get(urlPlace).text)['pages'][0]["segments"]
        placeDict = {}
        sectionPriceDict = {}
        sectionShapes = {}
        
        for p in rplace:
            placeDict[p['segments'][0]['id'].split(':')[1]] = p['name']
            sectionShapes[p['name']] = p['shapes'][0]['bounds'][0]

        for off in r2:
            try:
                section = int(placeDict[off["places"][0][:5]])
                if(not section in sectionPriceDict.keys()):
                    sectionPriceDict[section] = off['totalPriceRange'][0]['max']
                else:
                    sectionPriceDict[section] = min(sectionPriceDict[section],off['totalPriceRange'][0]['max'])
            except:
                pass
        totalListings = 1
        start = 0
        while(start < totalListings):
            urlsh="https://www.stubhub.ca/bfx/api/search/inventory/v2/listings?additionalPricingInfo=true&allSectionZoneStats=true&edgeControlEnabled=true&eventLevelStats=true&eventPricingSummary=true&listingAttributeCategorySummary=true&pricingSummary=true&quantitySummary=true&sectionStats=true&shstore=1&start="+str(start)+"&urgencyMessaging=true&valuePercentage=true&zoneStats=true&scoreVersion=v2&eventId="+str(eventIDstubhub[d['date']])+"&quantity=&rows=20&sort=price%20asc%2C%20value%20desc&priceType=bundledPrice&listingAttributeCategoryList=&excludeListingAttributeCategoryList=&deliveryTypeList=&sectionIdList=&zoneIdList=&pricemin=&pricemax=&listingRows="
            #urlsh = "https://www.stubhub.ca/bfx/api/search/inventory/v2/listings?additionalPricingInfo=true&allSectionZoneStats=true&edgeControlEnabled=true&eventLevelStats=true&eventPricingSummary=true&listingAttributeCategorySummary=true&pricingSummary=true&quantitySummary=true&sectionStats=true&shstore=1&start="+str(start)+"&urgencyMessaging=true&valuePercentage=true&zoneStats=true&scoreVersion=v2&eventId="+str(eventIDstubhub[d['date']])+"&quantity=&rows=20&sort=price%20asc%2C%20value%20desc&priceType=bundledPrice&listingAttributeCategoryList=&excludeListingAttributeCategoryList=&deliveryTypeList=&sectionIdList=60907&zoneIdList=&pricemin=&pricemax=&listingRows="
            rsh = requests.get(urlsh, auth=hawk_auth, headers=headers)
            stubhub = json.loads(rsh.text)
            totalListings = stubhub['totalListings']
            start += 20
            for l in stubhub['listing']:
                if(l['sectionName'][-3:].isnumeric()):
                    price = l['price']['amount']
                    section = int(l['sectionName'].split(' ')[-1])         
                    if(not section in sectionPriceDict.keys()):
                        sectionPriceDict[section] = price
                    else:
                        sectionPriceDict[section] = min(sectionPriceDict[section],price)

        fig = go.Figure()
        for k in sectionShapes.keys():
            pts = sectionShapes[k]
            ptx = [int(i.split(',')[0]) for i in pts.split(" ")]
            pty = [-int(i.split(',')[1]) for i in pts.split(" ")]
            if(int(k) in sectionPriceDict.keys()):
                price = sectionPriceDict[int(k)]
            else:
                price = 0
            color = get_color("Jet", ((price-50)/150*0.6)+0.4)
            if(price==0):
                color = "#555555"
            fig.add_trace(go.Scatter(x=ptx, y=pty, fill="toself",mode='lines',name="Section "+str(k)+" : "+str(price)+"CAD",  line=dict(color=color), showlegend=((price!=0)and ((price-50)/150*0.6)+0.4<.8)))
        fig.update_yaxes(
            scaleanchor = "x",
            scaleratio = 1,
          )
        fig.update_layout(
            title={
                'text': d['games'][0]['teams']['away']['team']['name']+" - " + my_format(d['date']),
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})
        fig.show()
        print(tlink)
        print()


https://www.ticketmaster.ca/event/31005B2A09713647?lang=fr-ca&brand=canadiens&landing=s&wt.mc_id=NHL_TEAM_MON_SCHEDULE_GAME4&lang=fr-ca



https://www.ticketmaster.ca/event/31005B2A09733649?lang=fr-ca&brand=canadiens&landing=s&wt.mc_id=NHL_TEAM_MON_SCHEDULE_GAME5&lang=fr-ca



https://www.ticketmaster.ca/event/31005B2A0974364B?lang=fr-ca&brand=canadiens&landing=s&wt.mc_id=NHL_TEAM_MON_SCHEDULE_GAME6&lang=fr-ca



https://www.ticketmaster.ca/event/31005B2A0976364E?lang=fr-ca&brand=canadiens&landing=s&wt.mc_id=NHL_TEAM_MON_SCHEDULE_GAME7&lang=fr-ca



https://www.ticketmaster.ca/event/31005B2A09783650?lang=fr-ca&brand=canadiens&landing=s&wt.mc_id=NHL_TEAM_MON_SCHEDULE_GAME8&lang=fr-ca



...

https://www.ticketmaster.ca/event/31005B2A0984366E?lang=fr-ca&brand=canadiens&landing=s&wt.mc_id=NHL_TEAM_MON_SCHEDULE_GAME16&lang=fr-ca



https://www.ticketmaster.ca/event/31005B2A09853670?lang=fr-ca&brand=canadiens&landing=s&wt.mc_id=NHL_TEAM_MON_SCHEDULE_GAME17&lang=fr-ca



https://www.ticketmaster.ca/event/31005B2A09873672?lang=fr-ca&brand=canadiens&landing=s&wt.mc_id=NHL_TEAM_MON_SCHEDULE_GAME18&lang=fr-ca



https://www.ticketmaster.ca/event/31005B2A09893674?lang=fr-ca&brand=canadiens&landing=s&wt.mc_id=NHL_TEAM_MON_SCHEDULE_GAME19&lang=fr-ca



https://www.ticketmaster.ca/event/31005B2A098A3678?lang=fr-ca&brand=canadiens&landing=s&wt.mc_id=NHL_TEAM_MON_SCHEDULE_GAME20&lang=fr-ca

